In [1]:
import operator
import pickle
import json
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import log_loss

from nltk.metrics.agreement import AnnotationTask

import matplotlib.pyplot as plt



In [2]:
def cross_entropy_loss(y_true, y_pred):
    try:
        return log_loss(y_true, y_pred, labels=[0,1])
    except ValueError:
        print(f'y_true {y_true}, {len(y_true)}')
        print(f'y_pred {y_pred}, {len(y_pred)}')        

In [3]:
def jaccard_similarity(l1, l2):  #expected two lists of words or two sets of words
    l1, l2 = set(l1), set(l2)
    intersection = l1.intersection(l2)
    union = l1.union(l2)
    try:
        return float(len(intersection)) / len(union)
    except ZeroDivisionError:
        return 0

In [4]:
def load_out_and_results_files(frame_name):
    frame_out_name = f'{frame_name}_out.pkl'
    frame_res_name =  f'{frame_name}_results.csv'
    frame_out = {}
    with open(f'{frame_out_name}', 'rb') as f:
        frame_out = pickle.load(f)
    frame_res = pd.read_csv(f'{frame_res_name}')
#     print(f'{frame_res.columns}')
    return frame_out, frame_res


In [5]:
#process frame_out's information to be used by calc score
def get_organized_frame_out(frame_out):
    #get features (words)
    organized = {'feats_pos': [], 'feats_neg': [], 'N_Chunks': []}
    for i, (ri, ca) in enumerate(zip(frame_out['raw_input_list'], frame_out['conti_attr_list'])):
        #subout beginnining <s> and end </s> tokens for ['BEGIN'] and ['END']
        raw_input_i = ['[BEGIN]'if f == '<s>' else '[END]' if f == '</s>' else f for f in ri]
        attr_appearance_cutoff = 5e-2
        ca = ca.to(torch.float32)
        
        #filtering out by zeroing non-appearing features
        ca_i = torch.where(torch.abs(ca) < attr_appearance_cutoff, torch.zeros(1), ca) 
        
        #get positive and negative features
        ca_i_pos = torch.where(ca_i > 0, ca_i, torch.zeros(1))
        ca_i_neg = torch.where(ca_i < 0, ca_i, torch.zeros(1))
        
        try:
            #get idx of pos/neg identified feature
            ca_i_pos_idx = torch.nonzero(ca_i_pos).squeeze().numpy()
            ca_i_neg_idx = torch.nonzero(ca_i_neg).squeeze().numpy() 
            #don't account for empty ''s or empty arrays
            features_pos = [raw_input_i[idx] for idx in ca_i_pos_idx if ri[idx] != ''] 
            features_neg = [raw_input_i[idx] for idx in ca_i_neg_idx if ri[idx] != ''] 
        except TypeError: #TypeError: iteration over a 0-d array
            #         print(f'i: {i}')
            #         print(f'{ri}')
            #         print(f'i: {raw_input_i}')
            #         print(f'{ca}')
            #         print(f'{ca_i}')
            #         print(f'{ca_i_idx}')
            #         print(f'features frame {features_frame}')
            #         print(f'N_Chunks {N_Chunks}')
            #         print(organized)
            features_pos = []
            features_neg = []
            

        
            
        organized['feats_pos'].append(features_pos)
        organized['feats_neg'].append(features_neg)
        
        N_cs = len(features_pos) + len(features_neg)
        organized['N_Chunks'].append(N_cs)
        
#      
    frame_out.update(organized)
    return frame_out
        

In [6]:
# process frame_res to be used by calc_score
def get_organized_frame_res(frame_res):
    organized = {'yh': [], 'non_neg': [], 'non_pos': [], 'should_neg': [], 'should_pos': [], 'trust_numbers': []}
    for x in frame_res.groupby('Input.qnli_number'):  # edit this groupby thing for different tasks
        task_answers = x[1]['Answer.taskAnswers']
        entailments, trusts = [], []
        non_red_ins, non_green_ins, fiat_red_ins, fiat_green_ins = [], [], [], []
        for answer_string in task_answers:  # edit this portion below to adapt
            json_obj = json.loads(answer_string)[0]
#             print(f'json obj {json_obj}')
            entailment = 1 if json_obj['entailment_radio']['1'] else 0
            entailments.append(entailment)
            trust = json_obj['trust_number'] if 'trust_number' in json_obj else 0         
            trusts.append(trust)
            
            if 'non_red_in' in json_obj.keys():
                non_red_ins.append([k.strip() for k in json_obj['non_red_in'].split(',')])
            else:
                non_red_ins.append([])

            if 'non_green_in' in json_obj.keys():
                non_green_ins.append([k.strip() for k in json_obj['non_green_in'].split(',')])
            else:
                non_green_ins.append([])

            if 'fiat_red_in' in json_obj.keys():
                fiat_red_ins.append([k.strip() for k in json_obj['fiat_red_in'].split(',')])
            else:
                fiat_red_ins.append([])

            if 'fiat_green_in' in json_obj.keys():
                fiat_green_ins.append([k.strip() for k in json_obj['fiat_green_in'].split(',')])
            else:
                fiat_green_ins.append([])

        organized['yh'].append(entailments)
        organized['non_neg'].append(non_red_ins)
        organized['non_pos'].append(non_green_ins)
        organized['should_neg'].append(fiat_red_ins)
        organized['should_pos'].append(fiat_green_ins)
        organized['trust_numbers'].append(trusts)
        
    # see agreement rate of turks:
    # assume 3 annotators
        
    annotation_triples = []
    for i, y_res in enumerate(organized['yh'], start=1):
        if len(y_res) != 3:
            print(f'i:{i}, don\'t have 3 answers')
            continue
        a1 = ('a1', str(i), y_res[0])
        a2 = ('a2', str(i), y_res[1])
        a3 = ('a3', str(i), y_res[2])
        annotation_triples.append(a1)
        annotation_triples.append(a2)
        annotation_triples.append(a3)      
    annotation_task = AnnotationTask(annotation_triples)
    average_ao = annotation_task.avg_Ao()

    return organized, average_ao

In [7]:
def calc_metr1_simulatability(yh, yg, ym, beta_1=1, beta_2=1):    
    loss_func = cross_entropy_loss
    l_yhyg = loss_func(yg, yh)
    l_yhym = loss_func(ym, yh)
    denom = beta_1 * l_yhyg + beta_2 * l_yhym + 1
    comp1 = (1/denom)
    return comp1

def metr1_wrapper(org_frame_out, org_frame_results):
    metr1s = []
    
    yhs = org_frame_results['yh']
    yms = np.round(org_frame_out['model_out_list'])
    ygs = org_frame_out['targets']
    yh_0, yh_1, yh_2 = [], [], [] #assume 3 annotaters
    for yh in yhs:
        if len(yh) != 3:
            print(f'len not 3 {yh}')
            continue 
        yh_0.append(yh[0])
        yh_1.append(yh[1])
        yh_2.append(yh[2])
    
    annotations = [yh_0, yh_1, yh_2]
    for yh in annotations:
        metr1s.append(calc_metr1_simulatability(yh, ygs, yms))
#     print(np.mean(metr1s), metr1s)
    return np.mean(metr1s), metr1s
    

In [8]:
def calc_metr2_fidelity(feat_f_pos, feat_f_neg, feat_h_pos, feat_h_neg):
    jaccard_pos = jaccard_similarity(feat_f_pos, feat_h_pos)
    jaccard_neg = jaccard_similarity(feat_f_neg, feat_h_neg)
    fidelity = np.mean([jaccard_pos, jaccard_neg])
#     print(f'pos jaccard {jaccard_pos}')
#     print(f'neg jaccard {jaccard_neg}')
#     print(f'fidelity {fidelity}')
    return fidelity 

def metr2_wrapper(org_frame_out, org_frame_results):
    metr2s = []
    for ff_pos, ff_neg, fh_nn_all, fh_np_all, fh_sn_all, fh_sp_all\
                            in zip(org_frame_out['feats_pos'], 
                               org_frame_out['feats_neg'], 
                               org_frame_results['non_neg'],
                               org_frame_results['non_pos'],
                               org_frame_results['should_neg'],
                               org_frame_results['should_pos']):
        metr2_annos = []
        for fh_nn, fh_np, fh_sn, fh_sp\
                            in zip(fh_nn_all, fh_np_all, fh_sn_all, fh_sp_all):
            fh_neg = set(ff_neg).difference(fh_nn).union(fh_sn)
            fh_pos = set(ff_pos).difference(fh_nn).union(fh_sp)
            
            metr2_an = calc_metr2_fidelity(ff_pos, ff_neg, fh_pos, fh_neg)
            metr2_annos.append(metr2_an)
        metr2s.append(metr2_annos)
    metr2s = np.array(metr2s)
    metr2s_average = metr2s.mean(axis=1).mean(axis=0)
    return metr2s_average, metr2s


In [9]:
def calc_metr3_complexity(N_c):
    if N_c == 0:
        return 0
    else:
        return 1/(np.log(N_c)+1)

def metr3_wrapper(org_frame_out, org_frame_results):
    metr3s = []
    N_Chunks = org_frame_out['N_Chunks']
    for N_c in N_Chunks:
        metr3 = calc_metr3_complexity(N_c)
        metr3s.append(metr3)
    metr3s_average = np.mean(metr3s)
#     print(metr3s_average, metr3s)
    return metr3s_average, metr3s


In [10]:
def calc_m1m2m3(org_frame_out, org_frame_results):
    m1, metric1s = metr1_wrapper(org_frame_out, org_frame_results)
    m2, metric2s = metr2_wrapper(org_frame_out, org_frame_results)
    m3, metric3s = metr3_wrapper(org_frame_out, org_frame_results)
    return m1, m2, m3

In [11]:
def score_wrapper(frame_names=['deep_lift', 'guided_backprop', 'input_x_gradients', 'integrated_gradients', 'kernel_shap', 'lime']):
    avg_aos = []
    m1s, m2s, m3s = [], [], []
    for frame_name in frame_names:
        print(f'Framework processed: {frame_name}')
        frame_out, frame_results = load_out_and_results_files(frame_name)
        org_frame_results, avg_ao = get_organized_frame_res(frame_results)
        avg_aos.append(avg_ao)
        org_frame_out = get_organized_frame_out(frame_out)    
        m1, m2, m3 = calc_m1m2m3(org_frame_out, org_frame_results)
        m1s.append(m1)
        m2s.append(m2)
        m3s.append(m3)
        print(f'm1: {m1:4f}, m2: {m2:4f}, m3: {m3:4f}')
    print(f'average average agreement {np.mean(avg_aos):.2f}')
    return frame_names, m1s, m2s, m3s

In [12]:
frame_names, m1s, m2s, m3s = score_wrapper(['deep_lift', 'guided_backprop', 'input_x_gradients', 'integrated_gradients', 'kernel_shap', 'lime'])

Framework processed: deep_lift
m1: 0.057034, m2: 0.859048, m3: 0.235495
Framework processed: guided_backprop
m1: 0.054720, m2: 0.866368, m3: 0.228977
Framework processed: input_x_gradients
m1: 0.055151, m2: 0.872433, m3: 0.235495
Framework processed: integrated_gradients
m1: 0.050538, m2: 0.793053, m3: 0.232914
Framework processed: kernel_shap
m1: 0.059342, m2: 0.854397, m3: 0.230393
Framework processed: lime
m1: 0.054025, m2: 0.850321, m3: 0.216652
average average agreement 0.65


In [13]:
def see_1_3_alpha(frame_names, m1s, m2s, m3s):
    for f, m1, m2, m3 in zip(frame_names, m1s, m2s, m3s):
        m1_13 = m1 * (1/3)
        m2_13 = m2 * (1/3)
        m3_13 = m3 * (1/3)
        score_13 = m1_13 + m2_13 + m3_13
        print(F"\\textbf{{f}} & {m1_13:.4f} & {m2_13:.4f} & {m3_13:.4f} & {score_13:.4f} \\\\ {f}")

see_1_3_alpha(frame_names, m1s, m2s, m3s)


\textbf{f} & 0.0190 & 0.2863 & 0.0785 & 0.3839 \\ deep_lift
\textbf{f} & 0.0182 & 0.2888 & 0.0763 & 0.3834 \\ guided_backprop
\textbf{f} & 0.0184 & 0.2908 & 0.0785 & 0.3877 \\ input_x_gradients
\textbf{f} & 0.0168 & 0.2644 & 0.0776 & 0.3588 \\ integrated_gradients
\textbf{f} & 0.0198 & 0.2848 & 0.0768 & 0.3814 \\ kernel_shap
\textbf{f} & 0.0180 & 0.2834 & 0.0722 & 0.3737 \\ lime


In [14]:
def generate_alpha_combos():
    import itertools
    alphas = np.arange(0, 11, step=1, dtype=np.uint8)
    all_combos = [(a/10,b/10,c/10) for (a,b,c) in itertools.product(alphas, alphas, alphas) if np.sum([a,b,c]) == 10]
    all_combos.reverse()
    return all_combos

In [15]:
alpha_combos = generate_alpha_combos()

In [16]:
def apply_alphas(frame_names, m1s, m2s, m3s, alpha_combos):
    scores_list = [] #indexed by the alpha combinations
    for combo in alpha_combos:
        a1, a2, a3 = combo[0], combo[1], combo[2]
        scores_for_fs = []
        for f, m1, m2, m3 in zip(frame_names, m1s, m2s, m3s):
            a1m1, a2m2, a3m3 = a1 * m1, a2 * m2, a3 * m3
            score = a1m1 + a2m2 + a3m3
            scores_for_fs.append(score)      
        scores_list.append([frame_names, scores_for_fs])
    return scores_list


In [17]:
scores_list = apply_alphas(frame_names, m1s, m2s, m3s, alpha_combos)

In [18]:
def normalize_score_list(scores_list):
    normalized_scores_list = []
    from sklearn.preprocessing import normalize
    def min_max_norm(scores):
        norm_scores = (scores-np.min(scores))/(np.max(scores)-np.min(scores))
        return norm_scores
    
    for (frame_names, scores) in scores_list:
        scores = np.array(scores)
#         normalized_scores = normalize([scores])[0]
#         normalized_scores = scores / np.sum(scores)
        normalized_scores = min_max_norm(scores)
#         print(f'after normalizing: {normalized_scores}\n-------------------------------------')
        normalized_scores_list.append([frame_names, normalized_scores])
    return normalized_scores_list

# scores_list = normalize_score_list(scores_list)  

In [19]:
def get_frame_scores(scores_list, alpha_combos):
    frame_names = scores_list[0][0]
    frame_scores = {}
    for i in range(len(frame_names)):
        frame_name = frame_names[i]
        frame_scores[frame_name] = [scores_list[j][1][i] for j in range(len(scores_list))]
    
   
    
    #sort frames based on average scores
    average_scores_dict = {}
    for frame, frame_scores_list in frame_scores.items():
        print(f'Average score for {frame}: {np.mean(frame_scores_list)}, std: {np.std(frame_scores_list)}')
        average_scores_dict[frame] = np.mean(frame_scores_list)
    sorted_average_scores_list =[[k, v] for k, v in sorted(average_scores_dict.items(), 
                                                          key=lambda item: item[1], reverse=True)]
    print(f'sorted average scores {sorted_average_scores_list}')

    ordered_frame_names = [k for [k,v] in sorted_average_scores_list]
    #sort based on top frame
    top_frame_name = ordered_frame_names[0]
    print(f'top_frame_name {top_frame_name}')
    sorted_idxs = np.argsort(frame_scores[top_frame_name])
    sorted_frame_scores = {}
    for frame in frame_names:
        sorted_frame_scores[frame] = np.array(frame_scores[frame])[sorted_idxs]
    
    permuted_alphas = np.array(np.array(alpha_combos)[sorted_idxs])
    a1_idx = np.where(permuted_alphas[:,0]==1)[0][0]
    a2_idx = np.where(permuted_alphas[:,1]==1)[0][0]
    a3_idx = np.where(permuted_alphas[:,2]==1)[0][0]
    
    
    return sorted_frame_scores, a1_idx, a2_idx, a3_idx, ordered_frame_names 

In [20]:
frame_scores, a1_idx, a2_idx, a3_idx, ordered_frame_names = get_frame_scores(scores_list, alpha_combos)

Average score for deep_lift: 0.38385919302479765, std: 0.19600612899591524
Average score for guided_backprop: 0.38335495793487806, std: 0.19888767450482805
Average score for input_x_gradients: 0.38769273437917373, std: 0.19986198806580951
Average score for integrated_gradients: 0.35883494044870007, std: 0.18011163446782238
Average score for kernel_shap: 0.38137741598965935, std: 0.1947917476373307
Average score for lime: 0.3736662240371027, std: 0.1958380147200183
sorted average scores [['input_x_gradients', 0.38769273437917373], ['deep_lift', 0.38385919302479765], ['guided_backprop', 0.38335495793487806], ['kernel_shap', 0.38137741598965935], ['lime', 0.3736662240371027], ['integrated_gradients', 0.35883494044870007]]
top_frame_name input_x_gradients


In [21]:
def output_graph_info(frame_scores, a1_idx, a2_idx, a3_idx, ordered_frame_names, task_name='QNLI'):
    out_dict = {
        'task_name': task_name,
        'frame_scores': frame_scores, 
        'a1_idx': a1_idx, 
        'a2_idx': a2_idx,
        'a3_idx': a3_idx,
        'ordered_frame_names': ordered_frame_names
    }
    out_file_name = f'{task_name}_graph.pkl'
    with open(out_file_name, 'wb') as f:
        pickle.dump(out_dict, f)
    return out_dict

In [22]:
output_graph_info(frame_scores, a1_idx, a2_idx, a3_idx, ordered_frame_names)

{'task_name': 'QNLI',
 'frame_scores': {'deep_lift': array([0.0570342 , 0.07488027, 0.09272634, 0.11057241, 0.12841848,
         0.13723563, 0.14626455, 0.1550817 , 0.16411062, 0.17292777,
         0.18195668, 0.19077384, 0.19980275, 0.20861991, 0.21764882,
         0.21743706, 0.22646597, 0.23549489, 0.23528313, 0.24431204,
         0.25312919, 0.26215811, 0.27097526, 0.28000418, 0.28882133,
         0.29785025, 0.29763848, 0.3066674 , 0.31548455, 0.32451347,
         0.33333062, 0.34235954, 0.35117669, 0.36020561, 0.36902276,
         0.37783991, 0.38686883, 0.39568598, 0.4047149 , 0.41353205,
         0.42256097, 0.43137812, 0.44922419, 0.45804134, 0.46707026,
         0.47588741, 0.48491633, 0.49373348, 0.51157955, 0.52942562,
         0.53824277, 0.54727169, 0.55608884, 0.57393491, 0.59178098,
         0.60962705, 0.6184442 , 0.63629027, 0.65413634, 0.67198241,
         0.69864563, 0.7164917 , 0.73433777, 0.77884706, 0.79669313,
         0.85904849]),
  'guided_backprop': array([0